# DemoStrategy

## 视频中介绍了针对四只股票的等权重投资策略，本段代码展示了利用quantOS系统进行策略回测及仿真交易的具体步骤。
## 在程序运行之前，需要您在环境设置中更改以下参数：
## 1. 将YourPhoneNo.改为您在quantOS网站注册的手机号;
## 2. 将YourToken改为您的token;
## 3. 将YourStrategyNo.改为您的策略号。

## 1. 环境设置

In [1]:
# -*- encoding: utf-8 -*-
import time
import pandas as pd
import numpy as np

from jaqs.data import RemoteDataService
from jaqs.trade import AlphaBacktestInstance
from jaqs.trade import PortfolioManager
#from jaqs.trade import RealTimeTradeApi

import jaqs.util as jutil
import jaqs.trade.analyze as ana
from jaqs.trade import AlphaStrategy
from jaqs.trade import AlphaTradeApi
from jaqs.trade import model
from jaqs.data import DataView

In [2]:
# 设置文件存储路径
dataview_dir_path = 'demoStrategy/dataview'
backtest_result_dir_path = 'demoStrategy'

# 设置服务器地址、用户名密码
# 例如：
# data_config = {
#   "remote.data.address": "tcp://data.tushare.org:8910",
#   "remote.data.username":  '18688888888',
#   "remote.data.password":  '23sdjfk209d0fs9dejkl2j3k4j9d0fsdf'}

# 如果没有使用quantos金融终端，请自行替换phone,token内容
import os
phone = os.environ.get("QUANTOS_USER")
token = os.environ.get("QUANTOS_TOKEN")

data_config = {
  "remote.data.address": "tcp://data.tushare.org:8910",
  "remote.data.username":  phone,
  "remote.data.password":  token}
trade_config = {
  "remote.trade.address": "tcp://gw.quantos.org:8901",
  "remote.trade.username":  phone,
  "remote.trade.password":  token}


# 设置Strategy number
# 例如：StrategyNo = 1043
StrategyNo = '1611'

## 2. 参数设置

In [3]:
# -------------------------------------------------------------------------------
# 设置目标股票、业绩基准、权重、时间
# -------------------------------------------------------------------------------
symbol_weights = {'600519.SH': 0.25,
                  '600036.SH': 0.25,
                  '601318.SH': 0.25,
                  '000651.SZ': 0.25}

benchmark = '000300.SH'

my_symbols = ','.join(symbol_weights.keys())
start_date = 20170201
end_date = 20171001

# 定义权重函数
def stockWeight(context, user_options=None):
    return pd.Series(symbol_weights)

## 3. 回测

In [4]:
# -------------------------------------------------------------------------------
# Main code 这个代码框不需要修改
# -------------------------------------------------------------------------------

def test_save_dataview():
    ds = RemoteDataService()
    ds.init_from_config(data_config)
    dv = DataView()

    props = {'start_date': start_date, 'end_date': end_date,
             'fields': 'sw1',
             'symbol': my_symbols,
             'freq': 1}

    dv.init_from_config(props, ds)
    dv.prepare_data()

    # set the benchmark
    res, _ = ds.daily(benchmark, start_date=dv.start_date, end_date=dv.end_date)
    dv._data_benchmark = res.set_index('trade_date').loc[:, ['close']]

    dv.save_dataview(folder_path=dataview_dir_path)


def test_alpha_strategy_dataview():
    dv = DataView()

    dv.load_dataview(folder_path=dataview_dir_path)

    props = {
        "symbol": dv.symbol,
        "universe": ','.join(dv.symbol),

        "start_date": dv.start_date,
        "end_date": dv.end_date,

        "period": "week",
        "days_delay": 0,

        "init_balance": 1e7,
        "position_ratio": 1.0,
        "commission_rate": 2E-4  # 手续费万2
    }
    props.update(data_config)
    props.update(trade_config)

    trade_api = AlphaTradeApi()

    signal_model = model.FactorSignalModel()
    signal_model.add_signal('stockWeight', stockWeight)

    strategy = AlphaStrategy(signal_model=signal_model, pc_method='factor_value_weight')
    pm = PortfolioManager()

    bt = AlphaBacktestInstance()
    
    context = model.Context(dataview=dv, instance=bt, strategy=strategy, trade_api=trade_api, pm=pm)
    
    signal_model.register_context(context)

    bt.init_from_config(props)

    bt.run_alpha()

    bt.save_results(folder_path=backtest_result_dir_path)
    

def test_backtest_analyze():
    ta = ana.AlphaAnalyzer()
    dv = DataView()
    dv.load_dataview(folder_path=dataview_dir_path)

    ta.initialize(dataview=dv, file_folder=backtest_result_dir_path)

    ta.do_analyze(result_dir=backtest_result_dir_path, selected_sec=ta.universe,
                  brinson_group=None)


In [6]:
# 运行这里跑回测
test_save_dataview()
test_alpha_strategy_dataview()
test_backtest_analyze()



Begin: DataApi login 15900631541@tcp://data.tushare.org:8910
    Already login as 15900631541, skip init_from_config
Initialize config success.
Query data...
Query data - query...
NOTE: price adjust method is [post adjust]
Query data - daily fields prepared.
Query instrument info...
Query adj_factor...
Query groups (industry)...


ValueError: No objects to concatenate

回测显示运行完成后，报告可从上面对话框最后一行的地址中找到 `HTML report: ...\demoStrategy\report.html`

## 4. 仿真交易

In [ ]:
from jaqs.trade.tradeapi import TradeApi

In [ ]:
tapi = TradeApi(trade_config['remote.trade.address'])

In [ ]:
def on_orderstatus(order):
    print("on_orderstatus:") #, order
    for key in order:    print("%20s : %s" % (key, str(order[key])))
    print("")


# 成交回报推送
def on_trade(trade):
    print("on_trade:")
    for key in trade:    print("%20s : %s" % (key, str(trade[key])))
    print("")

# 委托任务执行状态推送
# 通常可以忽略该回调函数
def on_taskstatus(task):
    print("on_taskstatus:")
    for key in task:    print("%20s : %s" % (key, str(task[key])))
    print("")

tapi.set_ordstatus_callback(on_orderstatus)
tapi.set_trade_callback(on_trade)
tapi.set_task_callback(on_taskstatus)

In [ ]:
user_info, msg = tapi.login(trade_config['remote.trade.username'], trade_config['remote.trade.password'])

In [ ]:
print(user_info)
tapi.use_strategy(StrategyNo)  #改成用户自己的 strategy号

res, msg = tapi.query_account()
print(res)

money = res.loc[0, 'enable_balance']
print("Balance we have: {}".format(money))

In [ ]:
dic_price      = {'600519.SH': 637.70,
                  '600036.SH': 28.72,
                  '601318.SH': 63.29,
                  '000651.SZ': 41.96}

# 每只股票等金额投资，各买入30万元
dic_shares = {k: 100 * np.floor(money * 0.012 * symbol_weights[k] / dic_price[k]/100) for k, _ in symbol_weights.items()}
dic_shares

orders = []
for symbol in symbol_weights.keys():
    o = {'security': symbol, 'price': dic_price[symbol], 'size': dic_shares[symbol], 'action': 'Buy'}
    orders.append(o)
orders

In [ ]:
# 买入股票
task_id, msg = tapi.place_batch_order(orders)
print(task_id)
print(msg)    

In [ ]:
# 做空一手沪深300股指期货做对冲
# task_id, msg = tapi.place_order("IF1712.CFE", "Short", 4003.6, 1)
# print(task_id)
# print(msg)